In [1]:
import sys
import os
# Get the current directory and parent directory
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.insert(0, parent_dir)
sys.path.insert(0, current_dir)

# Import from the-machine module
import importlib.util
the_machine_path = os.path.join(parent_dir, "the-machine")
sys.path.insert(0, the_machine_path)

# Import all from the-machine and custom_matroid_functions
from HM_module import *
from custom_matroid_functions import *

In [2]:
P = LaurentPolynomialRing(ZZ, 'x')
x = P.gen()
k, n = 4, 4
M = matroids.CompleteGraphic(n)
#M = matroids.Uniform(k, n)
L = M.lattice_of_flats()
r = M.rank()
rank_func = lambda x : M.rank(x)

#e = next(x for x in M.groundset() if x  not in M.coloops())
e = list(M.groundset())[0]
print(M.groundset())
print("deleting ", e)
mdele = M.delete(e)
Eq_M = create_deformed_algebra(M, base_ring=P)
Eq_mdele = create_deformed_algebra(mdele, base_ring=P)
E_mdele = Eq_mdele.natural()

E = Eq_M.natural()
E_basis = E.basis()
E_mdele_basis = E_mdele.basis()

C = Eq_M.C()
C_dele = Eq_mdele.C()
C_basis = C.basis()
C_dele_basis = C_dele.basis()

Z = Eq_M.zeta()
Z_dele = Eq_mdele.zeta()
Z_basis = Z.basis()

flats_keys = list(E_basis.keys())

frozenset({0, 1, 2, 3, 4, 5})
deleting  0


In [3]:
print("rank of M: ", r)
print("chow polynomial of M: ", chow_polynomial(M))
print(reduced_characteristic_polynomial(M))
C(E(flats_keys[-1]))

rank of M:  3
chow polynomial of M:  x^2 + 8*x + 1
x^2 - 5*x + 6


-(x+6*x^3)*C[frozenset()] + 2*x^2*C[frozenset({0})] - x*C[frozenset({0, 5})] - x*C[frozenset({0, 1, 3})] + 2*x^2*C[frozenset({4})] - x*C[frozenset({0, 2, 4})] - x*C[frozenset({3, 4, 5})] - x*C[frozenset({1, 4})] + 2*x^2*C[frozenset({5})] + 2*x^2*C[frozenset({1})] - x*C[frozenset({1, 2, 5})] + 2*x^2*C[frozenset({3})] + 2*x^2*C[frozenset({2})] - x*C[frozenset({2, 3})] + C[frozenset({0, 1, 2, 3, 4, 5})]

In [4]:
rev = {value: key for key, value in E_basis.items()}

def delta_flat(F):
    F_del_e = frozenset(set(F) - {e})
    return Eq_mdele(x^( - M.rank(F) + mdele.rank(F_del_e)) * E_mdele_basis[F_del_e])
    
def delta(elem):
    rep = E(elem)
    decom = {monomial: coeff for monomial, coeff in zip(rep.monomials(), rep.coefficients())}
    return Eq_mdele(sum(decom[F] * delta_flat(rev[F]) for F in decom))

dict = {F : C_dele(delta(C[F])) for F in E_basis.keys()}

In [5]:
def flat_division(M, e):
    L = M.lattice_of_flats()
    flat_div = {'S':set(), 'T':set(), 'A':set(), 'B':set()}
    for F in L:
        if e in F:
            if F - {e} in L:
                flat_div['T'].add(F)
            else:
                flat_div['A'].add(F)
        else:
            if F.union({e}) in L:
                flat_div['S'].add(F)
            else:
                flat_div['B'].add(F)
    return flat_div

In [6]:
from IPython.display import HTML, display
import pandas as pd

# Get the flat division
flat_div = flat_division(M, e)

# Create separate data lists for each category
data_s = []
data_t = []
data_a = []
data_b = []

basis_list_mdele = list(mdele.lattice_of_flats())

for F in dict:
    elem = dict[F]
    coeff_empty = elem.monomial_coefficients().get(frozenset(), 0)
    
    terms = []
    for S, coeff in elem.monomial_coefficients().items():
        terms.append(f"({coeff})*C[{str(set(S))}]")
    formatted_expr = " + ".join(terms) if terms else "0"
    
    row_data = {
        'F': str(set(F)),
        'rk(F)': M.rank(F),
        'delta(C[F])': formatted_expr,
        'coeff of empty': coeff_empty,
        'in zeta basis': str(Z_dele(elem))
    }
    
    # Categorize the flat and add to appropriate list
    if F in flat_div['S']:
        data_s.append(row_data)
    elif F in flat_div['T']:
        data_t.append(row_data)
    elif F in flat_div['A']:
        data_a.append(row_data)
    elif F in flat_div['B']:
        data_b.append(row_data)

# Create and display tables for each category
def display_table(data, category_name, color='lightblue'):
    if data:
        df = pd.DataFrame(data)
        display(HTML(f'<h3 style="color: {color};">Table for Category {category_name} ({len(data)} flats)</h3>'))
        display(HTML(
            f'<div style="height: 300px; overflow-y: auto; border: 2px solid {color}; padding: 10px;">{df.to_html(index=False)}</div>'
        ))
    else:
        display(HTML(f'<h3 style="color: {color};">Table for Category {category_name} (0 flats)</h3>'))
        display(HTML('<p>No flats in this category.</p>'))

# Display all four tables
display_table(data_s, 'S', 'green')
display_table(data_t, 'T', 'coral') 
display_table(data_a, 'A', 'blue')
display_table(data_b, 'B', 'red')

F,rk(F),delta(C[F]),coeff of empty,in zeta basis
set(),0,(1)*C[set()],1,Z[frozenset()]
{5},1,(1)*C[{5}],0,Z[frozenset({5})]


F,rk(F),delta(C[F]),coeff of empty,in zeta basis
{0},1,(x^-1 + x)*C[set()],x^-1 + x,(x^-1+x)*Z[frozenset()]
"{0, 5}",2,(x^-1 + x)*C[{5}],0,(x^-1+x)*Z[frozenset({5})]


F,rk(F),delta(C[F]),coeff of empty,in zeta basis
"{0, 2, 4}",2,"(1)*C[set()] + (1)*C[{2, 4}]",1,"Z[frozenset()] + Z[frozenset({2, 4})]"
"{0, 1, 3}",2,"(1)*C[set()] + (1)*C[{1, 3}]",1,"Z[frozenset()] + Z[frozenset({1, 3})]"
"{0, 1, 2, 3, 4, 5}",3,"(1)*C[{5}] + (1)*C[{1, 2, 3, 4, 5}]",0,"Z[frozenset({5})] + Z[frozenset({1, 2, 3, 4, 5})]"


F,rk(F),delta(C[F]),coeff of empty,in zeta basis
{2},1,(1)*C[{2}],0,Z[frozenset({2})]
{3},1,(1)*C[{3}],0,Z[frozenset({3})]
"{2, 3}",2,"(1)*C[{2, 3}]",0,"Z[frozenset({2, 3})]"
{1},1,(1)*C[{1}],0,Z[frozenset({1})]
"{1, 2, 5}",2,"(1)*C[{1, 2, 5}]",0,"Z[frozenset({1, 2, 5})]"
{4},1,(1)*C[{4}],0,Z[frozenset({4})]
"{1, 4}",2,"(1)*C[{1, 4}]",0,"Z[frozenset({1, 4})]"
"{3, 4, 5}",2,"(1)*C[{3, 4, 5}]",0,"Z[frozenset({3, 4, 5})]"


In [7]:
#\Delta(C[F]) - C[F\e] = 
def delta_diff(F):
    return delta(C(F)) -C_dele(F - {e})

print(E_basis.keys()[-1])
C_dele(delta_diff(E_basis.keys()[-1]))


frozenset({0, 1, 2, 3, 4, 5})


C[frozenset({5})]